In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\PC\Data\multimodal-rag-baseline


In [2]:
from configs.config import *
from libs.common import *
from utils.mongodb_connection import *
from utils.rag_utils import *
from utils.format_utils import *
load_dotenv(find_dotenv())

c:/Users/PC/Data/multimodal-rag-baseline


c:\ProgramData\miniconda3\envs\kyanon_digital\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
!nvidia-smi

Thu May  8 18:29:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   49C    P2             36W /  181W |    2584MiB /  16380MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch

print("GPU is available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)
    print("PyTorch is using GPU:", torch.cuda.current_device())
else:
    print("GPU is not available. PyTorch is running on CPU.")

GPU is available: True
GPU Name: NVIDIA GeForce RTX 4060 Ti
CUDA Version: 11.8
PyTorch is using GPU: 0


In [5]:
mongo_uri = os.getenv('MONGO_URI')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
collection = "kyanon_digital"

In [6]:
def extract_text_and_images_with_ocr(pdf_path):
    """
    Đọc nội dung và trích xuất văn bản từ ảnh trong PDF bằng OCR.

    Args:
        pdf_path (str): Đường dẫn đến tệp PDF.

    Returns:
        str: Nội dung văn bản từ PDF (bao gồm cả OCR từ ảnh).
    """
    full_text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        
        for page_number in range(len(pdf_document)):
            page = pdf_document[page_number]
            
            full_text += page.get_text()
            
            images = page.get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                
                image_filename = f"../data/img/temp_image_{page_number + 1}_{img_index + 1}.{image_ext}"
                with open(image_filename, "wb") as image_file:
                    image_file.write(image_bytes)
                
                image = Image.open(image_filename)
                ocr_text = pytesseract.image_to_string(image)
                full_text += ocr_text + "\n"
        
        return full_text
    except Exception as e:
        print(f"Lỗi khi xử lý PDF: {e}")
        return full_text

In [7]:
# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")
embedding_model = embedding_model.to('cuda')

print("Model is using GPU:", next(embedding_model.parameters()).is_cuda)

Model is using GPU: True


In [8]:
mongo_client = get_mongo_client(mongo_uri)
db = mongo_client['Kyanon']
collection = db['RAG']

Connection to MongoDB successful


In [9]:
def process_and_store_embeddings(pdf_path, mongo_client, chunk_size=500, overlap=10):
    """
    Quy trình: đọc PDF -> chia chunk -> tạo embedding -> lưu vào database.

    Args:
        pdf_path (str): Đường dẫn đến file PDF.
        mongo_client (str): Đường dẫn đến file SQLite database.
        chunk_size (int): Kích thước mỗi chunk.
        overlap (int): Độ chồng lấn giữa các chunk.
    """
    text_content = extract_text_and_images_with_ocr(pdf_path)

    chunks = split_text_into_chunks(text_content, chunk_size, overlap)
    print(chunks)
    print(f"Đã chia {len(chunks)} chunk từ văn bản.")

    chunk_embeddings = [(chunk, get_embedding(embedding_model, chunk)) for chunk in chunks]

    save_chunks_to_db(chunk_embeddings, mongo_client)

    print(f"Đã lưu {len(chunk_embeddings)} chunk và embedding vào database 'mongo_client'.")

In [10]:
pdf_path = os.path.join(root_dir, "data", "test.pdf").replace("\\", "/")
process_and_store_embeddings(pdf_path, mongo_client)

['2018 Baseball Hall of Fame balloting\n1 language\nArticle\nTalk\nRead\nEdit\nView history\nTools\n←\xa02017\n2019\xa0→\n2018 Baseball Hall of Fame\nballoting\n \n \n National Baseball \n \n \nHall of Fame and Museum\nNew inductees\n6\nvia BBWAA\n4\nvia Modern Baseball Era\nCommittee\n2\nTotal inductees\n323\nInduction date\nJuly 29, 2018\nKey\n\xa0\xa0\xa0\xa0\nElected to the Hall of Fame on this ballot (named\nin bold italics).\n\xa0\xa0\xa0\xa0\nElected subsequently, as of 2025 (named in plain\nitalics).\n\xa0\xa0\xa0\xa0\nRenominated for the 2019 BBWAA election by', "adequate performance on this ballot and has not\nsubsequently been eliminated.\n\xa0\xa0\xa0\xa0\nEliminated from annual BBWAA consideration by\npoor performance or expiration on subsequent\nFrom Wikipedia, the free encyclopedia\n(Redirected from Baseball Hall of Fame balloting, 2018)\nElections to the National Baseball Hall of Fame for 2018\nproceeded according to rules most recently amended in 2016.\nAs in the past,

In [12]:
query = "What is The BBWAA election rules?"
source_information = get_search_result(embedding_model, query.lower(), collection)
source_information

{'chunk': 'ballots.\n\xa0\xa0\xa0\xa0\nEliminated from annual BBWAA consideration by\npoor performance or expiration on this ballot.\n†\nFirst time on the BBWAA ballot.\n*\nEliminated from annual BBWAA consideration by\npoor performance on this ballot (not expiration).\nChipper Jones\nAnother recent rules change, announced in 2015,\ntightened the qualifications for the BBWAA\nelectorate. Beginning with the 2016 election,\neligible voters must not only have 10\xa0years of\ncontinuous BBWAA membership, but also be', 'score': 0.9589093923568726}
{'chunk': "elections. The BBWAA voted at the 2016 winter\nmeetings to make all Hall of Fame ballots public\none week after the results were announced,\neffective with this election.[5] However, the Hall\nrejected the BBWAA's proposal, and ballots remain\nconfidential unless individual voters choose to\nreveal them (which is allowed by Hall of Fame\nrules).[6]\nDonate Create account Log in", 'score': 0.9502994418144226}
{'chunk': '\u200b[ edit ]\nN

'\n 1) Content: ballots.\n\xa0\xa0\xa0\xa0\nEliminated from annual BBWAA consideration by\npoor performance or expiration on this ballot.\n†\nFirst time on the BBWAA ballot.\n*\nEliminated from annual BBWAA consideration by\npoor performance on this ballot (not expiration).\nChipper Jones\nAnother recent rules change, announced in 2015,\ntightened the qualifications for the BBWAA\nelectorate. Beginning with the 2016 election,\neligible voters must not only have 10\xa0years of\ncontinuous BBWAA membership, but also be\n 2) Content: elections. The BBWAA voted at the 2016 winter\nmeetings to make all Hall of Fame ballots public\none week after the results were announced,\neffective with this election.[5] However, the Hall\nrejected the BBWAA\'s proposal, and ballots remain\nconfidential unless individual voters choose to\nreveal them (which is allowed by Hall of Fame\nrules).[6]\nDonate Create account Log in\n 3) Content: \u200b[ edit ]\nNational Baseball Hall of Fame official website\nBB

In [13]:
combined_information = f"Become an expert on the concepts on Wikipedia. Customer's question: {query}\nAnswer the question based on the following document information: {source_information}."
print('==== LLMs Inference')
print(combined_information)

==== LLMs Inference
Hãy trở thành chuyên gia về các khái niệm trên wikipidea. Câu hỏi của khách hàng: What is The BBWAA election rules?
Trả lời câu hỏi dựa vào các thông tin tài liệu dưới đây: 
 1) Content: ballots.
    
Eliminated from annual BBWAA consideration by
poor performance or expiration on this ballot.
†
First time on the BBWAA ballot.
*
Eliminated from annual BBWAA consideration by
poor performance on this ballot (not expiration).
Chipper Jones
Another recent rules change, announced in 2015,
tightened the qualifications for the BBWAA
electorate. Beginning with the 2016 election,
eligible voters must not only have 10 years of
continuous BBWAA membership, but also be
 2) Content: elections. The BBWAA voted at the 2016 winter
meetings to make all Hall of Fame ballots public
one week after the results were announced,
effective with this election.[5] However, the Hall
rejected the BBWAA's proposal, and ballots remain
confidential unless individual voters choose to
reveal them (wh

## Gemini API model

In [14]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro')

In [15]:
%%time
response = model.generate_content(combined_information)
to_markdown(response.text)

CPU times: total: 93.8 ms
Wall time: 9.58 s


> The BBWAA (Baseball Writers' Association of America) election rules for the Hall of Fame, as of the information provided, are as follows:
> 
> * **Eligibility:** Players become eligible for the ballot five years after retirement.  The ballot includes players from the previous year's ballot who received at least 5% of the vote and haven't been eliminated, along with newly eligible players.  A screening committee selects players whose last MLB appearance was a certain number of years prior to the election (e.g., 2012 for the 2018 election).
> * **Voting Body:**  BBWAA members with 10 years of continuous membership and who are currently active or have been active within the past 10 years are eligible to vote.  Inactive members can regain voting privileges by covering MLB the year preceding the election.
> * **Ballot:** The ballot includes two categories of players as mentioned above.  The number of candidates on the ballot has varied over the years (e.g., reduced to eight from ten in a rule change mentioned in the provided text).
> * **Election Process:** The BBWAA members vote by mail. A player needs to receive 75% of the votes to be inducted into the Hall of Fame.
> * **Ballot Publicity:** While the BBWAA has attempted to make ballots public after the results are announced, the Hall of Fame itself has rejected this proposal. Ballots remain confidential unless individual voters choose to reveal them, which is allowed.
> * **Rules Changes:** The rules are subject to change. Several changes are mentioned in the provided text, including changes to the Veterans Committee (replaced by various panels over time), adjustments to the BBWAA voter eligibility requirements, and changes to the ballot composition process.
> 
> It's important to note that these rules may have been updated since the information provided was last updated (around 2018 based on the references).  For the most up-to-date information, it's always best to consult the official National Baseball Hall of Fame website and resources.


## Hugging Face model

In [16]:
from huggingface_hub import login

api_token = os.getenv('HUGGINGE_FACE_TOKEN') 
login(api_token)



In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]


In [18]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(response[0]))

<bos>Hãy trở thành chuyên gia về các khái niệm trên wikipidea. Câu hỏi của khách hàng: What is The BBWAA election rules?
Trả lời câu hỏi dựa vào các thông tin tài liệu dưới đây: 
 1) Content: ballots.
    
Eliminated from annual BBWAA consideration by
poor performance or expiration on this ballot.
†
First time on the BBWAA ballot.
*
Eliminated from annual BBWAA consideration by
poor performance on this ballot (not expiration).
Chipper Jones
Another recent rules change, announced in 2015,
tightened the qualifications for the BBWAA
electorate. Beginning with the 2016 election,
eligible voters must not only have 10 years of
continuous BBWAA membership, but also be
 2) Content: elections. The BBWAA voted at the 2016 winter
meetings to make all Hall of Fame ballots public
one week after the results were announced,
effective with this election.[5] However, the Hall
rejected the BBWAA's proposal, and ballots remain
confidential unless individual voters choose to
reveal them (which is allowed 